## What is Machine Translation?

When we hear about language translation, the first thing that comes to mind is Google Translate!!!, it's such a life savior. It has changed the world by allowing people to communicate when it wouldn’t otherwise be possible.<br>
Machine translation, sometimes referred to by the abbreviation MT is a sub-field of computational linguistics that investigates the use of software to translate text or speech from one language to another.(Source - Wikipedia)<br><br>
***
As famously quoted by the late Susan Sontag,<br>
> _"Translation is the circulatory system of the world's literatures"_

***
Machine translation is the task of automatically converting source text in one language to text in another language. Given a sequence of text in a source language, there is no one single best translation of that text to another language. This is because of the natural ambiguity and flexibility of human language. It is a challenging task that traditionally involves large statistical models developed using highly sophisticated linguistic knowledge.<br>
**Classical machine translation** methods often involve rules for converting text in the source language to the target language. The rules are often developed by linguists and may operate at the lexical, syntactic, or semantic level. This focus on rules gives the name to this area of study: Rule-based Machine Translation, or RBMT. The key limitations of the classical machine translation approaches are both the expertise required to develop the rules, and the vast number of rules and exceptions required.<br>


## What is Neural Machine Translation?

Neural machine translation, or NMT for short, is the use of neural network models to learn a statistical model for machine translation. The key benefit to the approach is that a single system can be trained directly on source and target text, no longer requiring the pipeline of specialized systems used in statistical machine learning.<br>
As such, neural machine translation systems are said to be _end-to-end systems_ as only one model is required for the translation.


### Encoder-Decoder Model

Multilayer Perceptron neural network models can be used for machine translation, although the models are limited by a fixed-length input sequence where the output must be the same length.<br>
These early models have been greatly improved upon recently through the use of recurrent neural networks(RNNs) organized into an encoder-decoder architecture that allow for variable length input and output sequences.<br>
As stated in [Neural Machine Translation by Jointly Learning to Align and Translate, 2014](https://arxiv.org/abs/1409.0473),
>An encoder neural network reads and encodes a source sentence into a fixed-length vector. A decoder then outputs a translation from the encoded vector. The whole encoder–decoder system, which consists of the encoder and the decoder for a language pair, is jointly trained to maximize the probability of a correct translation given a source sentence.

Key to the encoder-decoder architecture is the ability of the model to encode the source text into an internal fixed-length representation called the context vector. Interestingly, once encoded, different decoding systems could be used, in principle, to translate the context into different languages.